# Statistics
## Run statistics

In [1]:
%autoreload 2 
import numpy as np 
from pianoq.misc.mplt import mplot, mimshow
from pianoq.simulations.abstract_quantum_scaling.slm3_simulation import QWFSSimulation
from pianoq.misc.mplc_writeup_imports import * 


In [16]:
N_modes = 64
s = QWFSSimulation(N=N_modes)

# algos = ['slsqp', "L-BFGS-B", 'simulated_annealing', 'genetic_algorithm', 'PSO']
algos = ['slsqp', "L-BFGS-B", 'simulated_annealing']
# configs = ['SLM1', 'SLM2', 'SLM3']
configs = ['SLM1', 'SLM1-only-T']
T_methods = ['unitary', 'gaus_iid']

N_tries = 30

note = 'quick'
saveto_path = rf'G:\My Drive\Projects\MPLC\results\simulations\{tnow()}_qwfs_{note}.npz'
res = s.statistics(algos=algos, configs=configs, T_methods=T_methods, N_tries=N_tries, saveto_path=saveto_path)

# results = np.zeros((N_T_methods, N_configs, N_tries, N_algos))

# np.save(rf'C:\temp\{tnow()}_qwfs_results.npy', results)

try_no=0
try_no=1
try_no=2
try_no=3
try_no=4
try_no=5
try_no=6
try_no=7
try_no=8
try_no=9
try_no=10
try_no=11
try_no=12
try_no=13
try_no=14
try_no=15
try_no=16
try_no=17
try_no=18
try_no=19
try_no=20
try_no=21
try_no=22
try_no=23
try_no=24
try_no=25
try_no=26
try_no=27
try_no=28
try_no=29


## Show results 

In [31]:
print(np.pi/4)
print((np.pi/4)**2)

0.7853981633974483
0.6168502750680849


In [ ]:
res.print()

In [26]:
# comparing whether they find the same phases 
# mplot(best_phases[0, 2, 0, :3].T)